<a href="https://colab.research.google.com/github/NguyenThuan215/ML-echo-cardiography/blob/main/echocardiography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thư viện

In [11]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from google.colab import drive
import os

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

from collections import namedtuple
from sklearn.metrics import classification_report

import torch
from torch import nn
from torch.utils.data import DataLoader

# Kết nối với data lưu trong drive

In [12]:
drive.mount("/content/drive")
!unzip -uq "/content/drive/My Drive/DATA_CHAMBER_2021.zip" -d "./"
traindir = "DATA_CHAMBER_2021/train"
testdir = "DATA_CHAMBER_2021/test"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Chuẩn bị dữ liệu:
1. Các lớp: {2C, 3C, 4C}
2. Đọc dữ liệu trong file 'traindir' và 'testdir'
3. Đưa dữ liệu vào các batch để xử lý song song


In [13]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  image_size = 224
  transform_train = transforms.Compose([
      transforms.Resize((image_size,image_size)),                                    
      transforms.ToTensor(),
      # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  transform_test = transforms.Compose([
      transforms.Resize((image_size,image_size)),
      transforms.ToTensor(),
      # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test)

  print("Num Images in train set:", len(trainset))
  print("Num Images in test set:", len(testset))
  print("image size: ", trainset[0][0].shape)
  print("class: ", trainset.class_to_idx)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch = 32
  worker = 4
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch, shuffle=True, num_workers=worker)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch, shuffle=False, num_workers=worker)
  print("Num batch in train set: ", len(trainloader))
  return TrainTest(train=trainloader, test=testloader)

# Train/Test mô hình:
1. Train: với mỗi batch trong tập train:
  - Cho ảnh đi qua model
  - Tính lỗi bằng hàm lỗi "loss_func"
  - Cập nhật tham số
  - Báo cáo sau "reporting_steps" bước
2. Test:
  - Đặt model ở chế độ đánh giá (evaluate)
  - Tính toán đầu ra cho từng ảnh
  - trả về nhãn dự đoán/nhãn thực

In [14]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 40
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if step % reporting_steps == reporting_steps - 1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

# Tạo và thực nghiệm mô hình:
1. Tham số truyền vào main:
  - "PATH": file lưu lại mô hình
  - "model_in": string thể hiện tên mô hình muốn thực nghiệm.
2. Sửa đổi đầu ra của lớp Linear cuối cùng thành "3" để phù hợp với yêu cầu bào toán
3. Sử dụng hàm lỗi CrossEntropyLoss, hàm tối ưu SGD (Stochastic Gradient Descent)


In [15]:
def main(PATH='./model.pth', model_in=None):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print("\ndevice:", device)
  print("model:", model_in)

  if model_in == 'vgg16':  
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
  elif model_in == 'resnet50':
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True) 
  elif model_in == 'resnet18':
    model = torchvision.models.resnet18()
    model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=3, bias=True) 
  else: 
    pass


  model.to(device=device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  for epoch in range(20):
    print("---------------------------------------------------------------")
    print(f"\nEpoch {epoch} report: ")
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred_test, ytrue_test = test_epoch(epoch, model, loaders.test, device)
    print("Test report: \n", classification_report(ytrue_test, ypred_test, target_names=classes))
    
    torch.save(model.state_dict(), PATH)

  return model

In [20]:
model = main(PATH="./vgg16.pth", model_in='vgg16')

Num Images in train set: 6717
Num Images in test set: 1607
image size:  torch.Size([3, 224, 224])
class:  {'2C': 0, '3C': 1, '4C': 2}
Num batch in train set:  210

device: cuda
model: vgg16


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


---------------------------------------------------------------

Epoch 0 report: 
Epoch 0 step 39 ave_loss 1.0817
Epoch 0 step 79 ave_loss 1.0991
Epoch 0 step 119 ave_loss 1.0938
Epoch 0 step 159 ave_loss 1.0989
Epoch 0 step 199 ave_loss 1.0985


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test report: 
               precision    recall  f1-score   support

          2C       0.25      1.00      0.41       409
          3C       0.00      0.00      0.00       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.25      1607
   macro avg       0.08      0.33      0.14      1607
weighted avg       0.06      0.25      0.10      1607

---------------------------------------------------------------

Epoch 1 report: 
Epoch 1 step 39 ave_loss 1.0733
Epoch 1 step 79 ave_loss 1.0978
Epoch 1 step 119 ave_loss 1.0976
Epoch 1 step 159 ave_loss 1.0963
Epoch 1 step 199 ave_loss 1.0955
Test report: 
               precision    recall  f1-score   support

          2C       0.25      1.00      0.41       409
          3C       0.00      0.00      0.00       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.25      1607
   macro avg       0.08      0.33      0.14      1607
weighted avg      

...

Epoch 9 step 119 ave_loss 0.0097


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f340d405d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Epoch 9 step 159 ave_loss 0.0036


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f340d405d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Epoch 9 step 199 ave_loss 0.0006
Test report: 
               precision    recall  f1-score   support

          2C       0.76      0.94      0.84       409
          3C       0.88      0.95      0.91       367
          4C       1.00      0.85      0.92       831

    accuracy                           0.90      1607
   macro avg       0.88      0.91      0.89      1607
weighted avg       0.91      0.90      0.90      1607

---------------------------------------------------------------

Epoch 10 report: 
Epoch 10 step 39 ave_loss 0.0018
Epoch 10 step 79 ave_loss 0.0054
Epoch 10 step 119 ave_loss 0.0012
Epoch 10 step 159 ave_loss 0.0002
Epoch 10 step 199 ave_loss 0.0001
Test report: 
               precision    recall  f1-score   support

          2C       0.87      0.97      0.92       409
          3C       0.83      0.96      0.89       367
          4C       1.00      0.88      0.93       831

    accuracy                           0.92      1607
   macro avg       0.90      0.94

In [ ]:
model = main(PATH="./resnet18.pth", model_in='resnet18')

In [ ]:
model = main(PATH="./resnet50.pth", model_in='resnet50')

# Vẽ Biểu đồ

In [ ]:
# vgg16_64 = [0.25, 0.43, 0.66, 0.72, 0.80, 0.87, 0.90, 0.90, 0.90, 0.91]
# vgg16_224 = [0.25, 0.61, 0.78, 0.83, 0.86, 0.87, 0.88, 0.92, 0.89, 0.90]

# resnet18_64 = [0.56, 0.63, 0.75, 0.85, 0.89, 0.90, 0.90, 0.91, 0.90, 0.91]
# resnet18_224 = [0.65, 0.53, 0.71, 0.81, 0.83, 0.89, 0.90, 0.91, 0.90, 0.92]

# resnet50_64 = [0.65, 0.80, 0.79, 0.78, 0.80, 0.77, 0.83, 0.82, 0.82, 0.84]
# resnet50_224 = [0.61, 0.77, 0.83, 0.83, 0.86, 0.81, 0.83, 0.89, 0.85, 0.87]

# net = [vgg16_64, vgg16_224, resnet18_64, resnet18_224, resnet50_64, resnet50_224]
# fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2, figsize=(15,18))

# ax1.plot(vgg16_64, "bo--", label="64x64")
# ax1.set(title="VGG16 64x64", xlabel="epoch", ylabel="accuracy",xlim=(-0.5,10), ylim=(0,1))
# ax2.plot(vgg16_224, "go--", label="224x224")
# ax2.set(title="VGG16 224x224", xlabel="epoch", ylabel="accuracy", xlim=(-0.5,10), ylim=(0,1))

# ax3.plot(resnet18_64, "bo--", label="64x64")
# ax3.set(title="RESNET18 64x64", xlabel="epoch", ylabel="accuracy",xlim=(-0.5,10), ylim=(0,1))
# ax4.plot(resnet18_224, "go--", label="224x224")
# ax4.set(title="RESNET18 224x224", xlabel="epoch", ylabel="accuracy", xlim=(-0.5,10), ylim=(0,1))

# ax5.plot(resnet50_64, "bo--", label="64x64")
# ax5.set(title="RESNET50 64x64", xlabel="epoch", ylabel="accuracy",xlim=(-0.5,10), ylim=(0,1))
# ax6.plot(resnet50_224, "go--", label="224x224")
# ax6.set(title="RESNET50 224x224", xlabel="epoch", ylabel="accuracy", xlim=(-0.5,10), ylim=(0,1))

# fig.xlabel="x"
# fig.ylabel="y"
# plt.show()
